In [87]:

import pandas as pd
import os
import pdfplumber
import re
from datetime import datetime

In [101]:

data=[]
days_of_week=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
GACC=['AKCC', 'AICC', 'NWCC', 'ONCC', 'OSCC', 'NRCC', 'GBCC', 'SWCC', 'RMCC', 'EACC', 'SACC', 'AK', 'NW','NO', 'SO', 'NR', 'GB', 'SW', 'RM', 'EA', 'SA']
folder_path = "ISMR"
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]


errors=[]

pattern = re.compile(
    r'\b(?:Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday)\s*,?\s*(?:January|February|March|April|May|June|July|August|September|October|November|December)\s*\d{1,2}(?:st|nd|rd|th)?,?\s*\d{4}\s*[-–]\s*\d{4}\s*[A-Z]{2,4}\b'
)

#formats = [
#                
#                '%A %B %d, %Y',
#                '%A %B %dth, %Y',
#                '%A %B %drd, %Y',
#                '%A %B %dst, %Y',    # Format without comma
#
#                '%A, %B %d, %Y',
#                '%A, %B %dth, %Y',
#                '%A, %B %drd, %Y',
#                '%A, %B %dst, %Y', 
#
#                '%A , %B %d, %Y',
#                '%A , %B %dth, %Y',
#                '%A , %B %drd, %Y',
#                '%A , %B %dst, %Y', 
#            ]


#n=0
for file in files:

    file_path = f"ISMR/{file}"
    file_name = file.split('.')[0]
    date=None
    full_text = ""
    tables=[]


    with pdfplumber.open(file_path) as pdf:
        

        for page in pdf.pages:
    
            text = page.extract_text()
            if text:
                full_text += text + "\n"

        #    table = page.extract_table()
        #    tables.append(table)

            
    match = re.search(pattern, full_text) 
    try:
        date_str = match.group()
        cleaned = date_str.split('–')[0].strip()

        day=re.search(r'\d{1,2}', cleaned)
     
        year=re.search(r'\d{4}', cleaned)
  
        month=re.search(r'(?:January|February|March|April|May|June|July|August|September|October|November|December)', cleaned)
      
        date=year.group()+month.group()+day.group()
      
        date=datetime.strptime(date, '%Y%B%d')

        date = date.strftime('%Y-%m-%d')
       
    except AttributeError:
        errors.append(file_name)
        date='date parse error'
    

    a=0
    gacc_count=0
    for text_line in full_text.split('\n'):
        token = text_line.split()
        if not token:
            continue
        if token[0] in GACC:
                data.append([date]+token[0:7])
                a=1
                gacc_count+=1
        if gacc_count==10:
            break
    if a==0:
        errors.append(file_name)
    #n+=1
    #if n>500:
    #    break
                
                       





CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [102]:
errors

[]

In [103]:
data

[['2025-03-23', 'AICC', '0', '0', '0', '0', '0', '0'],
 ['2025-03-23', 'NWCC', '0', '0', '0', '0', '0', '0'],
 ['2025-03-23', 'ONCC', '0', '0', '0', '0', '0', '0'],
 ['2025-03-23', 'OSCC', '0', '0', '0', '0', '0', '0'],
 ['2025-03-23', 'NRCC', '0', '0', '0', '0', '0', '0'],
 ['2025-03-23', 'GBCC', '0', '0', '0', '0', '0', '0'],
 ['2025-03-23', 'SWCC', '3', '21,848', '2', '12', '0', '160'],
 ['2025-03-23', 'RMCC', '12', '49,237', '0', '70', '0', '285'],
 ['2025-03-23', 'EACC', '19', '8,296', '3', '31', '3', '243'],
 ['2025-03-23', 'SACC', '106', '266,809', '26', '234', '10', '1,212'],
 ['2016-11-26', 'AICC', '0', '0', '0', '0', '0', '0'],
 ['2016-11-26', 'NWCC', '0', '0', '0', '0', '0', '0'],
 ['2016-11-26', 'ONCC', '1', '1,235', '1', '0', '0', '6'],
 ['2016-11-26', 'OSCC', '0', '0', '0', '0', '0', '0'],
 ['2016-11-26', 'NRCC', '0', '0', '0', '0', '0', '0'],
 ['2016-11-26', 'GBCC', '0', '0', '0', '0', '0', '0'],
 ['2016-11-26', 'SWCC', '0', '518', '0', '0', '0', '0'],
 ['2016-11-26', 'R

In [104]:
df = pd.DataFrame(data,columns=['date','gacc','fires','cumulative_acres','crews','engines','helicopters','total_personnel'])
df.to_csv('active_incidents.csv')